In [99]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import DataFrame
import sys
from pymatgen.ext.matproj import MPRester
from pymatgen.core.structure import Structure
from pymatgen.analysis.structure_matcher import StructureMatcher
from ase import Atoms
from ase.io import write
from ase.visualize import view
from ase.visualize.ngl import view_ngl
from ase import Atoms
from ase.io import read, write
from ase.visualize.plot import plot_atoms
from ase.visualize import view
from ase.visualize.ngl import view_ngl,NGLDisplay
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.core.periodic_table import Element
import qmpy_rester as qr
# import nglview as ng
import copy
from tqdm import tqdm
from pymatgen.analysis.chemenv.coordination_environments.coordination_geometry_finder import LocalGeometryFinder,AbstractGeometry
from pymatgen.analysis.chemenv.coordination_environments.chemenv_strategies import SimplestChemenvStrategy
from pymatgen.analysis.chemenv.coordination_environments.structure_environments import LightStructureEnvironments
from pymatgen.analysis.chemenv.coordination_environments.structure_environments import StructureEnvironments
import os, sys
import numpy as np
import scipy
import argparse
from scipy.spatial import ConvexHull
from itertools import permutations
from pymatgen.core.structure import Structure
from pymatgen.core.periodic_table import *
from pymatgen.core.composition import *
from pymatgen.ext.matproj import MPRester
from pymatgen.io.vasp.outputs import *
from pymatgen.analysis.chemenv.coordination_environments.coordination_geometry_finder import LocalGeometryFinder
from pymatgen.analysis.chemenv.coordination_environments.structure_environments import LightStructureEnvironments
from pymatgen.analysis.chemenv.coordination_environments.chemenv_strategies import SimplestChemenvStrategy
from pymatgen.analysis.chemenv.coordination_environments.coordination_geometries import *

In [90]:
def plot_csm(struct_list,file_str_list=[],ehull_list=[]):
    
    ce_list = []
    isite_list = []
    isite = 0
    for i,eh in tqdm(zip(struct_list,ehull_list)):
#         print(file_str_list[struct_list.index(i)])
        lgf = LocalGeometryFinder()
        lgf.setup_structure(structure=i)
        if lgf!=[]:
#             se = lgf.compute_structure_environments(maximum_distance_factor=2,only_cations=True)
#             strategy = SimplestChemenvStrategy(distance_cutoff=1.4, angle_cutoff=0.3)
# #             print(se.structure)
#             lse = LightStructureEnvironments.from_structure_environments(strategy=strategy, structure_environments=se)
            m_specie_temp = ''
            num_anion = 0
            flag = 0
            for m in i:
                if str(m.specie) in ['Cl','O','S','F','Br','I','H']:
                    num_anion = num_anion+1
    #                 if m.specie!=anion_elements:
    #                     print(struct_list.index(i))
    #                     flag = 1
            volume = i.volume/num_anion
    #         if lse.coordination_environments[0] is not None:
            Li_list = []
#             print(i)
            for j in i:
                if str(j.specie) == 'Li':
                    isite = i.index(j)
                    Li_list.append(isite)
            for idx in Li_list:
                i_s = copy.deepcopy(i)
                Li_isites = copy.deepcopy(i.sites[idx])
                i_s.remove_species(['Li'])
                i_s.sites.insert(0,Li_isites)
                lgf = LocalGeometryFinder()
                lgf.setup_structure(structure=i_s)
                s_idx = struct_list.index(i)
                se = lgf.compute_structure_environments(maximum_distance_factor=1.5,only_cations=True,only_indices=[0],excluded_atoms=[
    'Sc','Ti','V','Cr','Mn','Fe','Co','Ni',
                        'Cu','Zn','Sr','Y','Zr','Er','In','Sn'
                       'Nb','Mo','La','Ce','Eu','Er',
                       'Hf','Ta','Pb','Mg','Sn',
    'Tc','Ru','Rh','Pd','Ag','Cd','In','Sn','Sb','Pr','Nd','Pm','Sm','Gd','Tb','Dy','Ho','P','Mg','Ca','Tm','Yb','Lu','W','Re','Os','Ir','Pt','Au','Hg','Tl'])
                strategy = SimplestChemenvStrategy(distance_cutoff=1.4, angle_cutoff=0.3)
                lse = LightStructureEnvironments.from_structure_environments(strategy=strategy, structure_environments=se)
#                 print(lse.coordination_environments[0])
                if lse.coordination_environments[0] is not None:
                    if len(lse.coordination_environments[0])>0:
                        csm_T_4 = se.get_csms(0,'T:4')
                        csm_O_6 = se.get_csms(0,'O:6')
                        if csm_T_4 != []:
                            lse.coordination_environments[0][0]['ce_symbol_I'] = 'T:4'
    #                            print(csm_T_4)
                            lse.coordination_environments[0][0]['ce_csm_I'] = csm_T_4[0]['symmetry_measure']
                        else:
                            lse.coordination_environments[0][0]['ce_symbol_I'] = 'T:4'
    #                            print(csm_T_4)
                            lse.coordination_environments[0][0]['ce_csm_I'] = None
                        if csm_O_6 != []:
                            lse.coordination_environments[0][0]['ce_symbol_T'] = 'O:6'
    #                            print(csm_O_6)
                            lse.coordination_environments[0][0]['ce_csm_T'] = csm_O_6[0]['symmetry_measure']
                        else:
                            lse.coordination_environments[0][0]['ce_symbol_T'] = 'O:6'
    #                            print(csm_T_4)
                            lse.coordination_environments[0][0]['ce_csm_T'] = None
    #                        if lse.coordination_environments[isite][0]['ce_symbol'] == 'T:4' or lse.coordination_environments[isite][0]['ce_symbol'] == 'O:6':
                        ce_list.append([lse.coordination_environments[0][0],volume,i.formula,eh,s_idx])
                    else:
                        pass

    return ce_list

In [100]:
def que_structure(x):
    with MPRester('niwEwLelQWb0kPBFE') as m:
        m_data = m.query(criteria = x, properties = ["unit_cell_formula",'e_above_hull', "pretty_formula","spacegroup","final_structure","band_gap",'elasticity','task_id']) #final_structure
    return m_data

In [92]:
def element_lists(e):
    element = []
    for i, j in enumerate(list(set(e.species))):
        element.append(str(j))
    return element

In [93]:
def str_with_non_anion_atoms(S):
    origin_elements = copy.deepcopy(element_lists(S))
    for i in ['O','S','Se','Te','F','Cl','Br','I','N']:
        # Br
        if i in origin_elements:
            origin_elements.remove(i)           
    S.remove_species(origin_elements)
    return S

In [94]:
def matching_anion_framework2(B):
    B1 = copy.deepcopy(B)
    s1 = str_with_non_anion_atoms(B1)
#     print(s1)
    fcc = Structure.from_file('../cif_structure/fcc.cif')
    bcc = Structure.from_file('../cif_structure/bcc.cif')
    hcp = Structure.from_file('../cif_structure/hcp.cif')
    m = StructureMatcher(ltol=0.2, stol=0.3, angle_tol=5, attempt_supercell=True, ignored_species=None)
#     fcc.make_supercell([4,4,4])
#     print(fcc)
    b = {}
#     view_Atoms(s1)
#     view_Atoms(bcc)
    c = {'fcc': fcc, 'bcc': bcc, 'hcp': hcp}
#     c = {'bcc':bcc}
    for i in ['bcc','fcc','hcp']:
        b[i] = m.fit_anonymous(s1, c[i])
    d = {}
    if True in b.values():
        for j in b.keys():
            if str(b[j]) == 'True':
                rms = m.get_rms_anonymous(s1, c[j])
                d['anion_type'] = j
                d['rms'] = round(rms[0],3)
    else:
        rms1 = m.get_rms_anonymous(s1, c['fcc'])        
        rms2 = m.get_rms_anonymous(s1, c['bcc'])
        rms3 = m.get_rms_anonymous(s1, c['hcp'])
        d['rms'] = [rms1,rms2,rms3]
        d['anion_type'] = 'none'
    return d

In [177]:
#!/usr/bin/env python
# This code extracts the lithium environment of all of lithium sites provided in a structure file.


'''
Input for the script : path to the structure file supported by Pymatgen
Structures with partial occupancy should be ordered or modified to full occupancy by Pymatgen.
'''
# parser = argparse.ArgumentParser()
# parser.add_argument('structure', help='path to the structure file supported by Pymatgen', nargs='?')
# parser.add_argument('envtype', help='both, tet, oct, choosing which perfect environment to reference to', nargs='?')
# args = parser.parse_args()


class HiddenPrints:
    '''
    class to reduce the output lines
    '''
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

def non_elements(struct, sp='Li'):
    '''
    struct : structure object from Pymatgen
    sp : the mobile specie
    returns the structure with all of the mobile specie (Li) removed
    '''
    num_li = struct.species.count(Element(sp))
    species = list(set(struct.species))
    species.remove(Element("O"))
    stripped = struct.copy()
    stripped.remove_species(species)
    stripped = stripped.get_sorted_structure(reverse=True)
    return stripped


def site_env(coord, struct, sp="Li", envtype='both'):
    '''
    coord : Fractional coordinate of the target atom
    struct : structure object from Pymatgen
    sp : the mobile specie
    envtype : This sets the reference perfect structure. 'both' compares CSM_tet and CSM_oct and assigns to the lower one.
    'tet' refers to the perfect tetrahedron and 'oct' refers to the perfect octahedron
    result : a dictionary of environment information
    '''
    stripped = non_elements(struct)
    with_li = stripped.copy()
    with_li.append(sp, coord, coords_are_cartesian=False, validate_proximity=False)
    with_li = with_li.get_sorted_structure()
    tet_oct_competition = []
    if envtype=='both' or envtype=='tet':
        for dist in np.linspace(1, 4, 601):
            neigh = with_li.get_neighbors(with_li.sites[0], dist)
            if len(neigh)<4:
                continue
            elif len(neigh)>4:
                break
            neigh_coords = [i.coords for i in neigh]
            with HiddenPrints():
                lgf = LocalGeometryFinder(only_symbols=["T:4"])
                lgf.setup_structure(structure=with_li)
                lgf.setup_local_geometry(isite=0, coords=neigh_coords)
            try:
                site_volume = ConvexHull(neigh_coords).volume
                tet_env_list = []
                for i in range(20):
                    tet_env = {'csm':lgf.get_coordination_symmetry_measures()['T:4']['csm'], 'vol':site_volume, 'type':'tet'}
                    tet_env_list.append(tet_env)
                tet_env = min(tet_env_list, key=lambda x:x['csm'])
                tet_oct_competition.append(tet_env)

            except Exception as e:
                print(e)
                print("This site cannot be recognized as tetrahedral site")
            if len(neigh)==4:
                break
    if envtype=='both' or envtype=='oct':
        for dist in np.linspace(1, 4, 601):
            neigh = with_li.get_neighbors(with_li.sites[0], dist)
            if len(neigh)<6:
                continue
            elif len(neigh)>6:
                break
            neigh_coords = [i.coords for i in neigh]
            with HiddenPrints():
                lgf = LocalGeometryFinder(only_symbols=["O:6"], permutations_safe_override=False)
                lgf.setup_structure(structure=with_li)
                lgf.setup_local_geometry(isite=0, coords=neigh_coords)
            try:
                site_volume = ConvexHull(neigh_coords).volume
                oct_env_list = []
                for i in range(20):
                    '''
                    20 times sampled in case of the algorithm "APPROXIMATE_FALLBACK" is used. Large number of permutations
                    are performed, but the default value in the function "coordination_geometry_symmetry_measures_fallback_random"
                    (NRANDOM=10) is often too small. This is not a problem if algorithm of "SEPARATION_PLANE" is used.
                    '''
                    oct_env = {'csm':lgf.get_coordination_symmetry_measures()['O:6']['csm'], 'vol':site_volume, 'type':'oct'}
                    oct_env_list.append(oct_env)
                oct_env = min(oct_env_list, key=lambda x:x['csm'])
                tet_oct_competition.append(oct_env)
                    
            except Exception as e:
                print(e)
                print("This site cannot be recognized as octahedral site")
            if len(neigh)==6:
                break

    if len(tet_oct_competition)==0:
        return {'csm':np.nan, 'vol':np.nan, 'type':'Non_'+envtype}
    elif len(tet_oct_competition)==1:
        return tet_oct_competition[0]
    elif len(tet_oct_competition)==2:
        csm1 = tet_oct_competition[0]
        csm2 = tet_oct_competition[1]
        if csm1['csm']>csm2['csm']:
            return csm2
        else:
            return csm1

def extract_sites(struct, sp="Li", envtype='both'):
    '''
    struct : structure object from Pymatgen
    envtype : 'tet', 'oct', or 'both'
    sp : target element to analyze environment

    '''
    envlist = []
    for i in range(len(struct.sites)):
        if struct.sites[i].specie != Element(sp):
            continue
        site = struct.sites[i]
        singleenv = site_env(site.frac_coords, struct, sp, envtype)
        envlist.append({'frac_coords':site.frac_coords, 'type':singleenv['type'], 'csm':singleenv['csm'], 'volume':singleenv['vol']})
    return envlist

def export_envs(envlist, sp='Li', envtype='both', fname=None,idx =None):
    '''
    envlist : list of dictionaries of environment information
    fname : Output file name
    '''
    if not fname:
        fname = "extracted_environment_info"+"_"+sp+"_"+envtype+".dat"
    with open(fname, 'w') as f:
        f.write('Structure id: '+idx+'\n')

        f.write('List of environment information\n')
        f.write('Species : '+sp+"\n")
        f.write('Envtype : '+envtype+"\n")
        for index, i in enumerate(envlist):
            f.write("Site index "+str(index)+": "+str(i)+'\n')



In [138]:
# element_ion = ['Li','Na']
element_anion = ['Se','O','Cl','Br','I','Fe','Co','Ni','Mn','Cr','S']
# element_trans_metal = [
#     'Sc','Ti','V','Cr','Mn','Fe','Co','Ni',
#                         'Cu','Zn','Sr','Y','Zr','Er','In','Sn'
#                        'Nb','Mo','La','Ce','Eu','Er',
#                        'Hf','Ta','Pb','Mg','Sn',
#     'Tc','Ru','Rh','Pd','Ag','Cd','In','Sn','Sb','Pr','Nd','Pm','Sm','Gd','Tb','Dy','Ho','P','Mg','Ca','Tm','Yb','Lu','W','Re','Os','Ir','Pt','Au','Hg','Tl']
# # 'Tc','Ru','Rh','Pd','Ag','Cd','In','Sn','Sb','Pr','Nd','Pm','Sm','Gd','Tb','Dy','Ho','P','Mg','Ca','Tm','Yb','Lu','W','Re','Os','Ir','Pt','Au','Hg','Tl',
# struct_list = []
# struct_dict = []
# for i in element_anion:
#     for j in element_trans_metal:
#         struct_list.append(i)
#         struct_list.append(j)
#         struct_dict.append(struct_list)
#         struct_list = []
structure_list = []
m_d = que_structure({"elements":{"$all":["Li","F"],"$nin":element_anion}, "nelements":{'$lte':4,'$gte':3}, "nsites":{'$lte':150,'$gte':2}})
# 'spacegroup.symbol':{"$in":['Pbca','Pnma']}
#     if m_d[0][]
#     print(m_d)



  0%|          | 0/633 [00:00<?, ?it/s]

In [139]:
ehull_list = []
id_list = []
for i in tqdm(m_d):
    structure_list.append(i['final_structure'])
    ehull_list.append(i['e_above_hull'])
    id_list.append(i['task_id'])


100%|█████████████████████████████████████| 633/633 [00:00<00:00, 146530.96it/s]


In [97]:
# structure_list[0].sites[0].specie
# [i for i in structure_list[0].types_of_species if i not in [Element('Li'),Element('O')]]

Element Li

In [136]:
for struct in tqdm(structure_list[0:-1]):
    idx =id_list[structure_list.index(struct)]
    site_info = extract_sites(struct, envtype='both')
    # print(site_info)
    for i in site_info:
        if i['csm']>2 and i['type']=='tet':
            os.chdir(f"/Users/runix/Nutstore Files/article_data/2022-09-05-Cl-distorted-structure/")
            subprocess.run(f'mkdir -p {idx}',shell=True)
            os.chdir(f"./{idx}/")
            struct.to(fmt='poscar',filename=f'{struct.formula}.vasp')
            struct.to(fmt='poscar',filename = f'POSCAR')
            os.chdir(f"../")
            export_envs(site_info, envtype='both',fname=f'/Users/runix/Nutstore Files/article_data/2022-09-05-Cl-distorted-structure/{idx}/{idx}.dat',idx=idx)
            os.chdir("../")
            continue
        else:
            pass
    # export_envs(site_info, envtype='both',fname=f'/Users/runix/Nutstore Files/article_data/2022-09-05-distorted-structure/{idx}.dat',idx=idx)


  6%|██▍                                       | 36/632 [01:11<23:31,  2.37s/it]

QH6154 Qhull precision error: Initial simplex is flat (facet 1 is coplanar with the interior point)

While executing:  | qhull i Qt
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 1989086648  incidence  Qtriangulate  _pre-merge  _zero-centrum
  _max-width 3.1  Error-roundoff 1.2e-14  _one-merge 8.7e-14
  _near-inside 4.4e-13  Visible-distance 2.5e-14  U-max-coplanar 2.5e-14
  Width-outside 5e-14  _wide-facet 1.5e-13  _maxoutside 1e-13

The input to qhull appears to be less than 3 dimensional, or a
computation has overflowed.

Qhull could not construct a clearly convex simplex from points:
- p3(v4):   2.4     9   8.2
- p1(v3): -0.76     9   8.2
- p2(v2):   2.4   8.6   5.6
- p0(v1): -0.76   8.6   5.6

The center point is coplanar with a facet, or a vertex is coplanar
with a neighboring facet.  The maximum round off error for
computing distances is 1.2e-14.  The center point, facets and distances
to the center point are as follows:

center point    0.804    8.824    6.896

facet 


  6%|██▌                                       | 39/632 [01:27<42:45,  4.33s/it]

QH6154 Qhull precision error: Initial simplex is flat (facet 1 is coplanar with the interior point)

While executing:  | qhull i Qt
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 1989355560  incidence  Qtriangulate  _pre-merge  _zero-centrum
  _max-width 3.2  Error-roundoff 1.2e-14  _one-merge 8.6e-14
  _near-inside 4.3e-13  Visible-distance 2.4e-14  U-max-coplanar 2.4e-14
  Width-outside 4.9e-14  _wide-facet 1.5e-13  _maxoutside 9.8e-14

The input to qhull appears to be less than 3 dimensional, or a
computation has overflowed.

Qhull could not construct a clearly convex simplex from points:
- p3(v4):  0.96   4.4   9.6
- p1(v3): -0.85   2.3   9.7
- p2(v2):  0.96   4.4   6.5
- p0(v1): -0.85   2.3   6.5

The center point is coplanar with a facet, or a vertex is coplanar
with a neighboring facet.  The maximum round off error for
computing distances is 1.2e-14.  The center point, facets and distances
to the center point are as follows:

center point  0.05411    3.332    8.072

fa


 10%|████                                      | 61/632 [05:58<49:36,  5.21s/it]

...


 77%|█████████████████████████████▊         | 484/632 [1:12:03<25:04, 10.17s/it]

QH6154 Qhull precision error: Initial simplex is flat (facet 1 is coplanar with the interior point)

While executing:  | qhull i Qt
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 2060550012  incidence  Qtriangulate  _pre-merge  _zero-centrum
  _max-width 3.1  Error-roundoff 1.4e-14  _one-merge 9.5e-14
  _near-inside 4.8e-13  Visible-distance 2.7e-14  U-max-coplanar 2.7e-14
  Width-outside 5.4e-14  _wide-facet 1.6e-13  _maxoutside 1.1e-13

The input to qhull appears to be less than 3 dimensional, or a
computation has overflowed.

Qhull could not construct a clearly convex simplex from points:
- p3(v4):   2.3   3.8    11
- p1(v3): -0.87   3.8    11
- p2(v2):   2.3   1.2    11
- p0(v1): -0.87   1.2    11

The center point is coplanar with a facet, or a vertex is coplanar
with a neighboring facet.  The maximum round off error for
computing distances is 1.4e-14.  The center point, facets and distances
to the center point are as follows:

center point   0.7028    2.506    10.63

fa


 93%|████████████████████████████████████   | 585/632 [1:17:23<02:56,  3.75s/it]

QH6154 Qhull precision error: Initial simplex is flat (facet 1 is coplanar with the interior point)

While executing:  | qhull i Qt
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 2065928252  incidence  Qtriangulate  _pre-merge  _zero-centrum
  _max-width 3.2  Error-roundoff 9.8e-15  _one-merge 6.9e-14
  _near-inside 3.4e-13  Visible-distance 2e-14  U-max-coplanar 2e-14
  Width-outside 3.9e-14  _wide-facet 1.2e-13  _maxoutside 7.9e-14

The input to qhull appears to be less than 3 dimensional, or a
computation has overflowed.

Qhull could not construct a clearly convex simplex from points:
- p3(v4):   3.5   4.7     6
- p0(v3):   1.2   1.6   4.9
- p2(v2):   3.6   3.5   3.9
- p1(v1):     1   2.8   7.1

The center point is coplanar with a facet, or a vertex is coplanar
with a neighboring facet.  The maximum round off error for
computing distances is 9.8e-15.  The center point, facets and distances
to the center point are as follows:

center point    2.339    3.142    5.478

facet 


100%|███████████████████████████████████████| 632/632 [1:21:21<00:00,  7.72s/it]


In [201]:
filename = '/Users/runix/Desktop/Li4MS4_phonon/LiM_MO3_2/La-Se/relax/'
struct = Structure.from_file(filename+str('POSCAR'))
site_info = extract_sites(struct, envtype='both')
# site_info
export_envs(site_info,fname=filename+str('envs.dat'),idx='mp-30249')

In [202]:
site_info

[{'frac_coords': array([0.30362567, 0.62571692, 0.18063141]),
  'type': 'tet',
  'csm': 6.860723778539381,
  'volume': 3.4761881093674263},
 {'frac_coords': array([0.44508437, 0.12299094, 0.81936437]),
  'type': 'tet',
  'csm': 6.861026520495484,
  'volume': 3.4758651056658803},
 {'frac_coords': array([0.37426066, 0.55491524, 0.6778935 ]),
  'type': 'tet',
  'csm': 6.858140071804314,
  'volume': 3.4772049510282126},
 {'frac_coords': array([0.87701769, 0.69636895, 0.32210427]),
  'type': 'tet',
  'csm': 6.858322766609085,
  'volume': 3.476411767601657}]

In [ ]:
import pickle
def save_variable(v,filename):
    f=open(filename,'wb')
    pickle.dump(v,f)
    f.close()
    return filename
def load_variavle(filename):
    f=open(filename,'rb')
    r=pickle.load(f)
    f.close()
    return r

In [ ]:
# import time
# t_begin = time.strftime('%Y-%m-%d-%H-%M-%S',time.localtime(time.time()))
# filename = save_variable(distorted_structure_list, f'distorted_structure_list-{t_begin}.txt')
# filename = save_variable(csm_t_1, f'csm_t_1-{t_begin}.txt')
# # str1 = Structure.from_file('../LMS_NEB/CONTCAR_Mn.vasp')

In [ ]:
# import os,subprocess
# os.chdir(f"/Users/runix/Nutstore Files/article_data/2022-09-05-distorted-structure/")
# for n,i in enumerate(distorted_structure_list[0:-1]):
#     file = os.listdir("/Users/runix/Nutstore Files/article_data/2022-09-05-distorted-structure/")
#     num = len(file)
#     subprocess.run(f'mkdir -p {n+num}',shell=True)
#     os.chdir(f"./{n+num}/")
#     i.to(fmt='poscar',filename=f'{i.formula}.vasp')
#     i.to(fmt='poscar',filename = f'POSCAR')
#     os.chdir(f"../")
# os.chdir("../")